In [1]:
import pandas as pd
import numpy as np
import xlwt
import xlrd
from tkinter import filedialog
import tkinter as tk
import customtkinter as ctk
from customtkinter import filedialog
import funciones
import openpyxl
import dbfread
import glob
from pandastable import Table, TableModel # Importar clases de pandastable


### cargar la tabla DBF

In [ ]:
archivo = None
while True:
    archivo=funciones.seleccionar_archivo()
    print(archivo)
    if archivo:
        ruta_archivo = archivo.name
        try:
            dbf = dbfread(ruta_archivo)
            liquidacion = pd.DataFrame(dbf)
            break
        except UnicodeDecodeError:
            print("Error de decodificación de caracteres. Intenta con otra codificación o archivo.")
            continue
    else:
        print("Archivo incorrecto. Por favor, vuelva a cargarlo.")

In [ ]:
liquidacion2 = funciones.subir_dbf()
print(liquidacion2)

In [ ]:
print(liquidacion2.head())

### cargar el conjunto de liquidaciones

In [ ]:
# Importar librerías

# Crear función para seleccionar archivos
def select_files():
    global files # Variable global para almacenar la lista de archivos
    files = filedialog.askopenfilenames(filetypes=[("Excel files", "*.xls")]) # Seleccionar solo archivos xlsx
    print(files) # Imprimir la lista de archivos

# Crear función para leer archivos y crear lista de dataframes
def read_files():
    global df_list # Variable global para almacenar la lista de dataframes
    df_list = [] # Crear lista vacía
    for file in files: # Iterar sobre cada archivo
        df = pd.read_excel(file) # Leer el archivo y saltar la primera fila
        df_list.append(df) # Añadir el dataframe a la lista
    print(df_list) # Imprimir la lista de dataframes

# Crear función para concatenar dataframes y crear uno nuevo
def concat_dfs():
    global df_final # Variable global para almacenar el dataframe final
    df_final = pd.concat(df_list, ignore_index=True) # Concatenar los dataframes de la lista y reiniciar el índice
    print(df_final) # Imprimir el dataframe final

# Crear función para mostrar dataframe con pandastable
def show_df():
    global table # Variable global para almacenar el objeto Table
    table = Table(frame, dataframe=df_final, visiblecols=True) # Crear objeto Table dentro del Frame
    table.show() # Mostrar la tabla en la ventana

# Crear interfaz gráfica
window = tk.Tk() # Crear ventana principal
window.title("Unir archivos excel") # Asignar título a la ventana
window.geometry("800x600") # Ancho 800, alto calculado


# Crear un widget Frame dentro de la ventana
frame = tk.Frame(window)
frame.pack()

# Crear botón para seleccionar archivos
btn_select = tk.Button(window, text="Seleccionar liquidaciones", command=lambda: [select_files(), read_files(), concat_dfs()])
btn_select.pack() # Colocar el botón en la ventana

# Crear botón para mostrar dataframe con pandastable
btn_show = tk.Button(window, text="Mostrar dataframe", command=show_df)
btn_show.pack() # Colocar el botón en la ventana

# Crear botón para seleccionar padrón DBF
btn_select = tk.Button(window, text="Seleccionar padrón", command= lambda:funciones.subir_dbf())
btn_select.pack() # Colocar el botón en la ventana

window.mainloop() # Iniciar el bucle principal de la ventana



In [ ]:
import pandas as pd
from tkinter import Tk, filedialog

root = Tk()
root.withdraw()

filenames = filedialog.askopenfilenames(title="Select Excel files")

df = pd.DataFrame()
for filename in filenames:
    temp_df = pd.read_excel(filename) 
    df = df.append(temp_df)

# Eliminar columnas duplicadas    
df.columns = df.columns.drop_duplicates() 

df.to_excel("merged.xlsx", index=False)